In [4]:
# load in saved multivariate_time_matrix i.e. our dataset
import torch
tensor = torch.load('multivariate_time_matrix.pt')

# cuda for gpu training
cuda_available = torch.cuda.is_available()
device = torch.device('cuda:0' if cuda_available else 'cpu')
torch.backends.cudnn.benchmark = True

from torch.utils.data import DataLoader
from torch.utils.data import Dataset

# from torch.utils.tensorboard import SummaryWriter
# Create a summary writer for logging with Tensorboard
# writer = SummaryWriter()

In [10]:
tensor.size()[0]

7537

In [ ]:
def padding(seqs, labels, inputDimSize, numClass):
    lengths = np.array([len(seq) for seq in seqs]) - 1
    maxlen = np.max(lengths)
    num_samples = len(seqs)
    

    x = torch.zeros(maxlen, num_samples, inputDimSize)
    y = torch.zeros(maxlen, num_samples, numClass)
    mask = torch.zeros(maxlen, num_samples)

    for idx, (seq, label) in enumerate(zip(seqs, labels)):
        for xvec, subseq in zip(x[:, idx, :], seq[:-1]):
            xvec[subseq] = 1.
        for yvec, subseq in zip(y[:, idx, :], label[1:]):
            yvec[subseq] = 1. 

        mask[:lengths[idx], idx] = 1.

    lengths = torch.LongTensor(lengths)
    return x, y, mask, lengths

In [ ]:
class patientDataset(Dataset):
    # tensor represents the time series data for one patient only
    # 42 (max number of visits) x 856 (number of unique icd code)
    def __init__(self, tensor):
        self.samples = tensor
    def __len__(self):
        # the last sample is not used
        return self.samples.size()[0]-1
    def __getitem(self, index):
        # Generate one sample of data from a patient
        X = self.sample[index,:]
        y = self.index[index+1,:]
        return X, y
    
# Parameters
params = {'batch_size': 5, 
          'shuffle': True;
          'num_workers': 6}

# for each iter of training loop
patient_set = patientDataset(tensor[0,:,:])
patient_generator = DataLoader(patient_set, **params)

        

In [ ]:
import argparse
parser = argparse.ArgumentParser(fromfile_prefix_chars='@')

parser_control = parser.add_argument_group('control options')
parser_control.add_argument('--infer',action='store_true')
parser_control.add_argument('--train',action='store_true')

parser_data = parser.add_argument_group('data options')
parser_data.add_argument('--data',default='names')

parser_model = parser.add_argument_group('model options')
parser_model.add_argument('--model',choices=['cnn','rnn','gru','lstm'],default='rnn')
parser_model.add_argument('--hidden_layer_size',type=int,default=128)
parser_model.add_argument('--num_layers',type=int,default=1)

parser_opt = parser.add_argument_group('optimization options')
parser_opt.add_argument('--batch_size',type=int,default=1)
parser_opt.add_argument('--learning_rate',type=float,default=1e-1)
parser_opt.add_argument('--optimizer',choices=['sgd','adam'],default='sgd')
parser_opt.add_argument('--gradient_clipping',action='store_true')
parser_opt.add_argument('--momentum',type=float,default=0.9)
parser_opt.add_argument('--weight_decay',type=float,default=1e-4)
parser_opt.add_argument('--samples',type=int,default=10000)
parser_opt.add_argument('--input_length',type=int,default=20)
parser_opt.add_argument('--warm_start')

parser_debug = parser.add_argument_group('debug options')
parser_debug.add_argument('--print_delay',type=int,default=5)
parser_debug.add_argument('--log_dir',type=str)
parser_debug.add_argument('--save_every',type=int,default=1000)

In [ ]:
# create features and labels by shifting data, specifically,
# exclude last row vector=visits for features, and exclude first row vector=0 for labels

# must do pca after this
def create_datasets(tensor):
    x_train = tensor[:,:]

# time series sequence prediction

# time series sequence classifcation

# time series sequence meta-learning, determine disease labels